-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Creating a Pseudonymized PII Lookup Table

In this lesson we'll create a pseudonymized key for storing potentially sensitive user data.

Our approach in this notebook is fairly straightforward; some industries may require more elaborate de-identification to guarantee privacy.

<img src="https://files.training.databricks.com/images/ade/ADE_arch_user_lookup.png" width="60%" />

## Learning Objectives
By the end of this lesson, students will be able to:
- Describe the purpose of "salting" before hashing
- Apply salted hashing to sensitive data for pseudonymization
- Use Auto Loader to process incremental inserts

## Setup
Begin by running the following cell to set up relevant databases and paths.

In [0]:
%run ../Includes/Classroom-Setup-6.1

Creating the database "dbacademy_odl_user_771624_databrickslabs_com_adewd_6_1"
Creating bronze...(4 seconds / 10,841,978 records)

Loading batch #0 to raw_user_reg...(2 seconds, 37 records)

Predefined Paths:
 DA.paths.working_dir: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/6.1
 DA.paths.user_db: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/6.1/6_1.db
 DA.paths.checkpoints: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/6.1/_checkpoints
 DA.paths.raw_user_reg: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/6.1/6_1.db/pii/raw_user_reg

Predefined tables in dbacademy_odl_user_771624_databrickslabs_com_adewd_6_1:
 bronze

Validating datasets...(1 seconds)

Setup completed in 10 seconds

## Auto Load Bronze Data

The following cell defines and executes logic to incrementally ingest data into the **`registered_users`** table with Auto Loader.

This logic is currently configured to process a single file each time a batch is triggered (currently every 10 seconds).

Executing this cell will start an always-on stream that slowly ingests new files as they arrive.

In [0]:
query = (spark.readStream
              .schema("device_id LONG, mac_address STRING, registration_timestamp DOUBLE, user_id LONG")
              .format("cloudFiles")
              .option("cloudFiles.format", "json")
              .option("cloudFiles.maxFilesPerTrigger", 1)
              .load(DA.paths.raw_user_reg)
              .writeStream
              .option("checkpointLocation", f"{DA.paths.checkpoints}/registered_users")
              .trigger(processingTime="10 seconds")
              .table("registered_users"))

DA.block_until_stream_is_ready(query)

The stream has processed 2 batchs

Before moving on with this lesson, we need to:
1. Stop the existing stream
2. Drop the table we created
3. Clear the checkpoint directory

In [0]:
query.stop()
query.awaitTermination()
spark.sql("DROP TABLE IF EXISTS registered_users")
dbutils.fs.rm(f"{DA.paths.checkpoints}/registered_users", True)

Out[28]: True

Use the cell below to refactor the above query into a function that processes new files as a single incremental triggered batch.

To do this:
* Remove the option limiting the amount of files processed per trigger (this is ignored when executing a batch anyway)
* Change the trigger type to "availableNow"
* Make sure to add **`.awaitTermination()`** to the end of your query to block execution of the next cell until the batch has completed

In [0]:
# TODO
def ingest_user_reg():
    query = (spark.readStream
              .schema("device_id LONG, mac_address STRING, registration_timestamp DOUBLE, user_id LONG")
              .format("cloudFiles")
              .option("cloudFiles.format", "json")
              #.option("cloudFiles.maxFilesPerTrigger", 1)
              .load(DA.paths.raw_user_reg)
              .writeStream
              .option("checkpointLocation", f"{DA.paths.checkpoints}/registered_users")
              #.trigger(processingTime="10 seconds")
              .trigger(availableNow=True)
              .table("registered_users"))
    query.awaitTermination()

Use your function below.

**NOTE**: This triggered batch will automatically cause our always-on stream to fail because the same checkpoint is used; default behavior will allow the newer query to succeed and error the older query.

In [0]:
ingest_user_reg()
display(spark.table("registered_users"))

device_id,mac_address,registration_timestamp,user_id
145943,c7:12:d1:03:f1:76,1.557989967E9,27703
148312,34:12:9e:9c:72:28,1.561870112E9,28521
161343,e6:de:fe:e2:2f:4d,1.562995114E9,28776
193806,53:b5:20:22:c4:4b,1.564375794E9,28938
183816,e5:a8:d5:73:cf:33,1.562010799E9,29213
113024,72:2f:58:79:1b:6a,1.56430927E9,30088
113570,46:30:9c:da:50:a7,1.560507319E9,30612
172633,d5:ed:0b:e5:b7:00,1.555539598E9,31362
182045,f7:a1:8d:26:c4:4b,1.563832178E9,32018
126126,e4:2c:d4:ea:e4:b4,1.559688692E9,33987


Another custom class was initialized in the setup script to land a new batch of data in our source directory. 

Execute the following cell and note the difference in the total number of rows in our tables.

In [0]:
DA.user_reg_stream.load()

ingest_user_reg()
display(spark.table("registered_users"))

Loading batch #1 to raw_user_reg...(1 seconds, 5 records)

device_id,mac_address,registration_timestamp,user_id
145943,c7:12:d1:03:f1:76,1.557989967E9,27703
148312,34:12:9e:9c:72:28,1.561870112E9,28521
161343,e6:de:fe:e2:2f:4d,1.562995114E9,28776
193806,53:b5:20:22:c4:4b,1.564375794E9,28938
183816,e5:a8:d5:73:cf:33,1.562010799E9,29213
113024,72:2f:58:79:1b:6a,1.56430927E9,30088
113570,46:30:9c:da:50:a7,1.560507319E9,30612
172633,d5:ed:0b:e5:b7:00,1.555539598E9,31362
182045,f7:a1:8d:26:c4:4b,1.563832178E9,32018
126126,e4:2c:d4:ea:e4:b4,1.559688692E9,33987


## Add a Salt to Natural Key
We'll start by defining a salt, here in plain text. We'll combine this salt with our natural key, **`user_id`**, before applying a hash function to generate a pseudonymized key.

Salting before hashing is very important as it makes dictionary attacks to reverse the hash much more expensive.  To demonstrate, try reversing the following SHA-256 hash of `Secrets123` by searching it's hash using Google: `FCF730B6D95236ECD3C9FC2D92D7B6B2BB061514961AEC041D6C7A7192F592E4` bringing you to this [link](https://hashtoolkit.com/decrypt-sha256-hash/fcf730b6d95236ecd3c9fc2d92d7b6b2bb061514961aec041d6c7a7192f592e4).  Next, try hashing `Secrets123:BEANS` [here](https://passwordsgenerator.net/sha256-hash-generator/) and perform a similar search.  Notice how adding the salt `BEANS` improved the security.

For greater security, we could upload the salt as a secret using the Databricks <a href="https://docs.databricks.com/security/secrets/secrets.html" target="_blank">Secrets</a> utility.

In [0]:
salt = 'BEANS'
spark.conf.set("da.salt", salt)

In [0]:
# # If using the Databricks secrets store, here's how you'd read it...
# salt = dbutils.secrets.get(scope="DA-ADE3.03", key="salt")
# salt

Preview what your new key will look like.

In [0]:
%sql
SELECT *, sha2(concat(user_id,"${da.salt}"), 256) AS alt_id
FROM registered_users

device_id,mac_address,registration_timestamp,user_id,alt_id
145943,c7:12:d1:03:f1:76,1.557989967E9,27703,c7d80aeb1a9e7653dc13dadec6075e3e2a32f0226bfeac1968248b4c7164d8ab
148312,34:12:9e:9c:72:28,1.561870112E9,28521,cba874842f4dd76d361c730e6c6080f130d86f149c1f6faeaf3d0dc605c85d17
161343,e6:de:fe:e2:2f:4d,1.562995114E9,28776,8bb09f205d023251e392804a20a7294f5a424c9c295054c8f8f293c76dd205a5
193806,53:b5:20:22:c4:4b,1.564375794E9,28938,0fabdd33422fa07ecacd79b4c8d0799897a404b8eb52b23996c2b40cf3b600bc
183816,e5:a8:d5:73:cf:33,1.562010799E9,29213,bf89f14b9b61eb3369399515152f55c944de897791ff42e5d1ef9305ad2da394
113024,72:2f:58:79:1b:6a,1.56430927E9,30088,886f1bdac99a41378ec91389e91c1f4d8033e5cd61084223d9ca95807d812a55
113570,46:30:9c:da:50:a7,1.560507319E9,30612,9ac4a92a8ad072a8c613708fa3dada52f430ead19466922536403464f16c5f48
172633,d5:ed:0b:e5:b7:00,1.555539598E9,31362,8a5876de067c1386bdb74084a16d2c7f2f18fbdee589ffe73c0ceb4f38eb5922
182045,f7:a1:8d:26:c4:4b,1.563832178E9,32018,c65fc3b744666216a6d3f3d23686cbcd95c823368c132f05314d9ecac587b170
126126,e4:2c:d4:ea:e4:b4,1.559688692E9,33987,8e53b301a9819419b49d4315fe9c95fd752e2d72a94820f89f1acd4d14eb560f


## Register a SQL UDF

Create a SQL user-defined function to register this logic to the current database under the name **`salted_hash`**. 

This will allow this logic to be called by any user with appropriate permissions on this function. 

Make sure your UDF accepts one parameter: a **`String`** and it should return a **`STRING`**.  You can access the configured salt value by using the expression `"${da.salt}"`.

For more information, see the <a href="https://docs.databricks.com/spark/latest/spark-sql/language-manual/sql-ref-syntax-ddl-create-sql-function.html" target="_blank">CREATE FUNCTION</a> method from the SQL UDFs docs.

In [0]:
%sql
-- TODO
CREATE OR REPLACE FUNCTION salted_hash (id STRING) RETURNS STRING
RETURN sha2(concat(id,"${da.salt}"), 256)

If your SQL UDF is defined correctly, the assert statement below should run without error.

In [0]:
# Check your work
set_a = spark.sql(f"SELECT sha2(concat('USER123', '{salt}'), 256) alt_id").collect()
set_b = spark.sql("SELECT salted_hash('USER123') alt_id").collect()
assert set_a == set_b, "The 'salted_hash' function is returning the wrong result."
print("All tests passed.")

All tests passed.

Note that it is theoretically possible to link the original key and pseudo-ID if the hash function and the salt are known.

Here, we use this method to add a layer of obfuscation; in production, you may wish to have a much more sophisticated hashing method.

## Register Target Table
The logic below creates the **`user_lookup`** table.

Here we're just creating our **`user_lookup`** table. In the next notebook, we'll use this pseudo-ID as the sole link to user PII.

By controlling access to the link between our **`alt_id`** and other natural keys, we'll be able to prevent linking PII to other user data throughout our system.

In [0]:
%sql
CREATE TABLE IF NOT EXISTS user_lookup (alt_id string, device_id long, mac_address string, user_id long)
USING DELTA 
LOCATION '${da.paths.working_dir}/user_lookup'

## Define a Function for Processing Incremental Batches

The cell below includes the setting for the correct checkpoint path.

Define a function to apply the SQL UDF registered above to create your **`alt_id`** to the **`user_id`** from the **`registered_users`** table.

Make sure you include all the necessary columns for the target **`user_lookup`** table. Configure your logic to execute as a triggered incremental batch.

In [0]:
# TODO
def load_user_lookup():
    (spark.readStream
        .schema("alt_id STRING, device_id LONG, mac_address STRING, user_id LONG")
        .format("cloudFiles")
        .option("cloudFiles.format", "json")
        .load(DA.paths.registered_users)
        .writeStream
        .option("checkpointLocation", f"{DA.paths.checkpoints}/user_lookup")
        .trigger(availableNow=True)
        .table("user_lookup"))
    )

File "<command-4382774100892662>" , line 12 
 ) 
 ^
 SyntaxError : unmatched ')'

Use your method below and display the results.

In [0]:
load_user_lookup()

display(spark.table("user_lookup"))

Process another batch of data below to confirm that the incremental processing is working through the entire pipeline.

In [0]:
DA.user_reg_stream.load()

ingest_user_reg()
load_user_lookup()

display(spark.table("user_lookup"))

The code below ingests all the remaining records to put 100 total users in the **`user_lookup`** table.

In [0]:
DA.user_reg_stream.load(continuous=True)

ingest_user_reg()
load_user_lookup()

display(spark.table("user_lookup"))

We'll apply this same hashing method to process and store PII data in the next lesson.

Run the following cell to delete the tables and files associated with this lesson.

In [0]:
DA.cleanup()

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>